# LVM Spectrograph Lab photon rate

Nick Konidaris, 25-2-2021

We sent a Hg lamp through a fiber system and detected it with a calibrated photodiode. Here I compute the predicted detected signal on the back end of the LVM spectrograph and compare it to the desired signal level.


In [1]:
import numpy as np
import astropy
from astropy import units as u
from astropy import constants as k

from pylab import *

In [2]:
mm = u.mm
μw = 1e-6*u.watt
c = k.c
h = k.h

hc = h*c

π = np.pi

Pavan measured the output of the Thorlabs 2" integrating sphere. He concluded that it has a 40x attenuation at the 404 nm line of mercury.

The photodiode output indicated a signal of (0.1163-0.0115) micro Watts with a PD, after QE correciton, with a photodiode that is 8.72 mm in diameter. This comes from the LVM-I Illumination system google sheet.

In [3]:
D_diode = 8.72*mm

# The radiant flux of the 404 line is measured directly by the PD
radiant_flux = (0.1163 - 0.115)*μw

# The radiance incident on the photodiode has a solid angle of 2π (it's inside an integrating sphere) and an area set by the PD
L_eΩ = radiant_flux / (2*π*u.sr) / (π * (D_diode/2)**2)

epp = hc/(404 * u.nanometer)

In [4]:

# Radiance in photon units
L_γΩ = (L_eΩ/epp).to(u.Hz/u.sr/u.cm**2)

In [5]:
print("Photon radiance is: %s" % L_γΩ)

Photon radiance is: 704604261.4487697 Hz / (cm2 sr)


## 4-f system accounting

Now we have to account for the 4-f system. Let's say it has a throughput of 50%.

In [6]:
def fno_to_solid_angle(fno):
    """ Compute the solid angle given a focal ratio """
    return π/(2*fno)**2 * u.sr

In [7]:
d_fiber = 107 * u.micron

In [8]:
η_4f = 0.5

A_fiber = π*(d_fiber/2)**2
grasp_4f = fno_to_solid_angle(3.5) * A_fiber * η_4f

In [9]:
grasp_4f

<Quantity 288.25791017 micron2 sr>

In [10]:
L_γΩ_out_4f = (L_γΩ * grasp_4f).to(u.Hz)
print("Photon rate on fiber tip: %s" % L_γΩ_out_4f)

Photon rate on fiber tip: 2031.0775190467273 Hz


## Spectrogaph System accounting

In [11]:
η_spectrograph = 0.3 
L_γΩ_det = L_γΩ_out_4f * η_spectrograph

print("Photon rate detected at 404 line: %s" % L_γΩ_out_4f)

Photon rate detected at 404 line: 2031.0775190467273 Hz


In [12]:
GOAL = 300000 # We need 100,000 photons in a line for a super robust detection

etime = (GOAL/L_γΩ_det).to(u.min)

print("An exposure time for a robust signal is %s" % (etime))

An exposure time for a robust signal is 8.205824992090433 min
